In [30]:
# %load ../start2.py
# Imports
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
sys.path.insert(0, '../../lib/python')
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs
from ncbi_remap.io import add_data_columns, remove_id, add_id

# Setup notebook
nbconfig = Nb.setup_notebook()

# Turn on cache
from joblib import Memory
memory = Memory(cachedir=nbconfig.cache, verbose=0)

# Connect to data store
store = pd.HDFStore('../../sra.h5', mode='a')

last updated: 2017-11-26 
Git hash: d93d3e9dec278c9175123b44f16939fc03a6ef06


I had a set of SRRs that fastq-dump was timing out on. I have decided to remove them here and can revisit in the future.

In [ ]:
# List of srrs that were giving me problems.
srrs = [
    'SRR1171029',  'SRR1171036',  'SRR1171039',  'SRR1211256',  'SRR1525685',  'SRR1525694', 
    'SRR1525695',  'SRR1525696',  'SRR1525698',  'SRR1525768',  'SRR1525769',  'SRR1525771', 
    'SRR1525772',  'SRR1525774',  'SRR1688626',  'SRR1769729',  'SRR1988509',  'SRR1988510', 
    'SRR1988512',  'SRR1988513',  'SRR1988514',  'SRR1988515',  'SRR1988517',  'SRR1988518', 
    'SRR1994856',  'SRR2006603',  'SRR2044310',  'SRR2071689',  'SRR2071692',  'SRR2071698', 
    'SRR2071708',  'SRR2071731',  'SRR2071744',  'SRR2071745',  'SRR2071750',  'SRR3939051', 
    'SRR3939058',  'SRR3939091',  'SRR3939099',  'SRR514297',  'SRR514298',  'SRR514299', 
    'SRR514300',  'SRR514301',  'SRR514302',  'SRR543922',  'SRR543924',  'SRR543927', 
    'SRR543929',  'SRR543931',  'SRR543932',  'SRR543934',  'SRR543937',  'SRR834512', 
    'SRR835219',  'SRR835221',  'SRR835223',  'SRR835228',  'SRR835236',  'SRR835242', 
    'SRR835247',  'SRR835252',  'SRR835256',  'SRR835326',  'SRR835329',  'SRR835333', 
    'SRR835338',  'SRR835341',  'SRR835343',  'SRR835345',  'SRR835347',  'SRR835349', 
    'SRR835939',  'SRR835942',  'SRR933565',  'SRR933571',  'SRR933572',  'SRR933579'
]

In [28]:
# Get srrs from queue to get their SRXs

try:
    bad = store.select('prealn/queue', 'srr == srrs')
except NameError:
    add_data_columns(store, 'prealn/queue')
    bad = store.select('prealn/queue', 'srr == srrs')

bad.shape

(78, 2)

In [29]:
pattern = '../../prealn-wf/output/samples/{srx}/{srr}/DOWNLOAD_BAD'

In [41]:
for _, row in bad.iterrows():

    # Create flag to indicate bad download
    flagBad = Path(pattern.format(srx=row.srx, srr=row.srr))
    flagBad.touch()

    # Update store
    remove_id(store, 'prealn/queue', srx=row.srx, srr=row.srr)
    add_id(store, 'prealn/download_bad', srx=row.srx, srr=row.srr)

In [33]:
# Reverse the process
#for _, row in bad.iterrows():
#
#    # Create flag to indicate bad download
#    flagBad = Path(pattern.format(srx=row.srx, srr=row.srr))
#    flagBad.unlink()
#
#    # Update store
#    remove_id(store, 'prealn/download_bad', srx=row.srx, srr=row.srr)
#    add_id(store, 'prealn/queue', srx=row.srx, srr=row.srr)

In [42]:
store.close()